# Byte-Wise Markov Chain Models for Language Detection
This project is a response to the Fellowship.AI Language Detection challenge. The prompt is:


> [European Parliament Proceedings Parallel Corpus](http://www.statmt.org/europarl/) is a text dataset used for evaluating language detection engines. The 1.5GB corpus includes 21 languages spoken in EU.  

> Create a machine learning model trained on this dataset to predict the following [test set](https://storage.googleapis.com/google-code-archive-downloads/v2/code.google.com/language-detection/europarl-test.zip).

## LSTM Aspirations

I had been working with deep neural network models and really wanted to train up an LSTM. I was inspired by a paper published by Open AI, [Learning to Generate Reviews and Discovering Sentiment](https://arxiv.org/pdf/1704.01444.pdf), where the researchers train an unsupervised "[Byte mLSTM](https://arxiv.org/pdf/1609.07959.pdf)" model on Amazon reviews and the network [discovers sentiment on its own](https://blog.openai.com/unsupervised-sentiment-neuron/) (with no sentiment labels involved.) This was a fascinating result. I wanted to apply this technique to the Europarl corpus to build a language detector. Surely, if the Byte mLSTM can learn something as subtle as sentiment without supervision, it can learn to tell apart langauges.<br><br>
I put together [some code](https://github.com/AlliedToasters/language_neurons) using this [mLSTM repository](https://github.com/guillitte/pytorch-sentiment-neuron) and rented a Google Cloud Compute instance (using my free $300 credit) to take advantage of a Tesla K80 GPU-enabled machine. I cleaned up the data a little bit and started training my mLSTM. This was Saturday night, and as of this writing it is now Wednesday and 84 hours have passed. The machine is roughly through 15 percent of A SINGLE EPOCH, even with full CUDA optimization with PyTorch. Perhaps I shouldn't be surprised, since the original researchers mention that "training took one month across four NVIDIA Pascal GPUs..." on a corpus larger than Europarl. The code on GitHub seems to suggest the training was completed in 10 epochs.

### Time to Think...
With my LSTM chunking its way through the corpus, I had a lot of time to think. I wanted to simplify the model so it wouldn't take so long. Maybe something I could run on my laptop without GPUs.

## Modeling Byte Sequences
I spent some time learning about UTF-8 encoding (the encoding used in the Europarl corpus). Unlike ASCII, UTF-8 uses a variable number of bytes per character. It is reverse-compatible with ASCII, and reserves the ASCII bytes for a one-byte-per-character scheme. To expand its vocabulary to encompass all of Unicode, UTF-8 uses certain bytes (with decimal values above 128) to signal a coming byte chain that will encode a single unicode character.<br><br>
As many of the languages in this corpus use unique characters, a model than can look at the sequences of bytes that appear in a given language will be powerful.

## Markov Chains
A Markov Chain model assumes that the probability of the next link of the series (in our case, the next byte) depends only on the previous links (the previous bytes.) The number of bytes in the model's "memory" is known as the "order" of the model.<br><br>
This is a first-order Markov Chain model of the sentence, "One fish two fish red fish blue fish."

![First-Order Markov Chain](https://cdn-images-1.medium.com/max/1600/1*UuD1-B7CFn3M2nUDy02sHg.png)

Notice the first-order model gives equal probability to the next state being "two", "red", "blue", or \*END\* when the current state is "fish."<br><br>
To improve the model, we can increase the order of the model to remember the state or states before the word Fish, giving us a way to model the order of the words "two", "red", and "blue." (Thanks to [Alexander Dejeu and Hackernoon for the image](https://hackernoon.com/from-what-is-a-markov-model-to-here-is-how-markov-models-work-1ac5f4629b71).)

## Markov Chains for Classification
Markov Chain models are usually used to predict the next character or word in a series. A famous example is the "next word recommendation" feature when writing a text message. These simple models work surprisingly well in this application, helping some users compose texts more quickly.<br><br>
However, in this case, we are looking to classify chains of bytes as coming from a certain language. To do this, we just need to sum the probabilities of a set of pre-trained models moving along the byte sequence and apply the softmax function to the outputs values to get likelihoods for each class.


## Preparing the Data
I will train with the Europarl corpus and validate with the set provided by Fellowship AI.

In [1]:
#Hide training data in git-ignored ./data directory because it's too big for GitHub.
!mkdir data
!wget -P ./data http://www.statmt.org/europarl/v7/europarl.tgz
!tar -xzf ./data/europarl.tgz -C ./data/
#Download and unzip validation data
!wget https://storage.googleapis.com/google-code-archive-downloads/v2/code.google.com/language-detection/europarl-test.zip
!unzip europarl-test.zip
#Install dependencies
!pip install scipy
!pip install numpy
!pip install pandas
!pip install sparse

'\n!mkdir data\n!wget -P ./data http://www.statmt.org/europarl/v7/europarl.tgz\n!tar -xzf ./data/europarl.tgz -C ./data/\n!pip install pandas\n'

In [2]:
import pandas as pd
from markov_model import update_progress
import os
import re
import sys

In [3]:
#Create a dataframe to manage files and labels.
cols = ['lang', 'path']
df = pd.DataFrame(columns=cols)

languages = os.popen('ls ./data/txt/').read().split('\n')[:-1]
for lang in languages:
    print('adding language {}...'.format(lang))
    lang_frame = pd.DataFrame(columns=cols)
    txts = os.popen('ls ./data/txt/{}/'.format(lang)).read().split('\n')[:-1]
    paths = ['./data/txt/{}/{}'.format(lang, txt) for txt in txts]
    lang_frame['path'] = paths
    lang_frame['lang'] = lang
    lang_frame.index = range(len(df), len(df)+len(lang_frame))
    df = pd.concat([df, lang_frame], axis=0)

adding language bg...
adding language cs...
adding language da...
adding language de...
adding language el...
adding language en...
adding language es...
adding language et...
adding language fi...
adding language fr...
adding language hu...
adding language it...
adding language lt...
adding language lv...
adding language nl...
adding language pl...
adding language pt...
adding language ro...
adding language sk...
adding language sl...
adding language sv...


## Labelled Text
To simplify training, I combine all of the texts for each language into a single .txt file located at ./data/{language}.txt. I remove some html tag-looking artifacts from the dataset.

In [4]:
def get_text(idx, df):
    """Takes idx and df and returns text."""
    path = df.loc[idx].path
    with open(path, 'r') as f:
        result = f.read()
    return result

def remove_tags(input_string):
    """Removes html tag-like artifacts in the data."""
    result = input_string
    tag = re.compile(r'<[^<]*>')
    while tag.search(result):
        match = tag.search(result)
        strt = match.span()[0]
        stp = match.span()[1]
        result = result[:strt] + result[stp:]
    return result

for lang in df.lang.unique():
    cnt = 0
    full_text = ''
    lng_df = df[df.lang == lang]
    print('gathering data: ', lang)
    for i, row in lng_df.iterrows():
        try:
            txt = get_text(i, lng_df)
        except UnicodeDecodeError:
            print('problem decoding: ', i, lang)
            continue
        txt = remove_tags(txt)
        full_text += txt
        cnt += 1
        prog = cnt/len(lng_df)
        update_progress(prog)
    with open('./data/{}.txt'.format(lang), 'w+', encoding='utf-8') as f:
        f.write(full_text)

gathering data:  bg
Progress: [#########################] 100% Done...
gathering data:  cs
Progress: [#########################] 100% Done...
gathering data:  da
Progress: [#########################] 100% Done...
gathering data:  de
Progress: [#########################] 100% Done...
gathering data:  el
Progress: [#########################] 100% Done...
gathering data:  en
Progress: [#########################] 100% Done...
gathering data:  es
Progress: [#########################] 100% Done...
gathering data:  et
Progress: [#########################] 100% Done...
gathering data:  fi
Progress: [#########################] 100% Done...
gathering data:  fr
Progress: [#########################] 100% Done...
gathering data:  hu
Progress: [#########################] 100% Done...
gathering data:  it
Progress: [#########################] 100% Done...
gathering data:  lt
Progress: [#########################] 100% Done...
gathering data:  lv
Progress: [#########################] 100% Done...
gather

Note there is one decoding error in the language "pl." Since this problem occurs just once, I will omit the text from that file from the training data.

## Markov Chain Software
It was surprisingly hard to find software to apply this model. I ended up coding  